In [25]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt 
import seaborn as sns
import os
import importlib
import download_clean_data as dc
import ipca
import metrics
import kernel_regression as kr
importlib.reload(dc) 
importlib.reload(ipca)
importlib.reload(metrics)
importlib.reload(kr)

<module 'kernel_regression' from 'c:\\Users\\dandr_wh38grr\\Documents\\GitHub\\ML-project-2\\kernel_regression.py'>

In [26]:
folder_path = "Data/monthly_data"

In [27]:
N = 100
date = 20150000
data, ret = dc.download_clean_data(folder_path, date, N)

In [28]:
x = []
for t in range(len(ret)-1):
    x.append(data[t].values.T@ret[t+1].values/len(ret[0]))

x_cov = np.sum([x[i].reshape((-1,1))@x[i].reshape((1,-1)) for i in range(len(x))], axis = 0)
eigval_x, eigvec_x = np.linalg.eig(x_cov)

idx = np.argsort(np.abs(eigval_x))[::-1]
sort_eigvec_x = eigvec_x[:,idx].real
k = 5
gamma_first = sort_eigvec_x[:,:k]

In [29]:
max_iter = 100
gamma, f_list = ipca.ipca(data, ret, gamma_first.copy(), max_iter)
W = np.eye(N)
W_list = [W]*(len(data)-1)
lambda1 = 0.1
lambda2 = 0.1
gamma_reg_w, f_list_reg_w = ipca.ipca_reg_w(data, ret, gamma_first.copy(), max_iter, lambda1, lambda2, W_list)

In [30]:
print(metrics.total_R_squared(ret, data, gamma, f_list),metrics.pred_R_squared(ret, data, gamma, f_list))

0.15335785516821054 -0.004563129725073445


In [31]:
print(metrics.total_R_squared(ret, data, gamma_reg_w, f_list_reg_w),metrics.pred_R_squared(ret, data, gamma_reg_w, f_list_reg_w))

0.15167256959390385 -0.0028241664513473097


In [32]:
def split_dataset(x,y, trsh):

    n = int(np.floor(len(y)*trsh))

    x_train=x[:n]
    y_train=y[:n]
    x_test=x[n:]
    y_test=y[n:]
        
    return x_train,y_train,x_test,y_test

In [41]:
def cross_val_IPCA(y,x, trsh, gamma_first, max_iter):

    total_R2_dict = {}
    pred_R2_dict = {}
    
    xx_train,yy_train,xx_test,yy_test = split_dataset(x,y, trsh)

    gamma, _ = ipca.ipca(xx_train, yy_train, gamma_first.copy(), max_iter)
    print('done')

    yy_pred = []

    for i in range(len(xx_test)-1):

        f = ipca.solve_f(ret, data, gamma, i+1)
        yy_pred.append(f)

    total_R2_dict[('IPCA')] = metrics.total_R_squared(yy_test, xx_test, gamma, yy_pred)
    pred_R2_dict[('IPCA')] = metrics.pred_R_squared(yy_test, xx_test, gamma, yy_pred)

    return total_R2_dict, pred_R2_dict

In [36]:
def cross_val_IPCA_reg(y,x, trsh, lambda1_v, lambda2_v, gamma_first, max_iter):

    total_R2_dict = {}
    pred_R2_dict = {}
    
    xx_train,yy_train,xx_test,yy_test = split_dataset(x,y, trsh)

    for lambda1 in lambda1_v:
        for lambda2 in lambda2_v:

            gamma_reg_w, _ = ipca.ipca_reg_w(xx_train, yy_train, gamma_first.copy(), max_iter, lambda1, lambda2, W_list)

            yy_pred = []

            for i in range(len(xx_test)-1):

                f = ipca.solve_f_reg_w(yy_test, xx_test, gamma_reg_w, i+1, lambda1, W_list[i])
                yy_pred.append(f)

            total_R2_dict[('IPCA_reg', lambda1, lambda2)] = metrics.total_R_squared(yy_test, xx_test, gamma_reg_w, yy_pred)
            pred_R2_dict[('IPCA_reg', lambda1, lambda2)] = metrics.pred_R_squared(yy_test, xx_test, gamma_reg_w, yy_pred)

    return total_R2_dict, pred_R2_dict

In [37]:
y = ret
x = data
trsh = 0.8
lambda1_v = list([0.1])
lambda2_v = list([0.1])

cross_val_IPCA_reg(y,x, trsh, lambda1_v, lambda2_v, gamma_first)

({('IPCA_reg', 0.1, 0.1): 0.024975152887454444},
 {('IPCA_reg', 0.1, 0.1): 0.0005013007171978945})

In [42]:
cross_val_IPCA(y,x, 0.9, gamma_first)

KeyboardInterrupt: 

In [9]:
data2 = data.copy()
data2 = np.array(np.array(data2).reshape(72*100,94)) #flatten data, build X
tk = 1
l = 1
K = kr.K_LR(data2, tk, l)

In [10]:
Omega1=np.eye((len(data)-1)*N)
Omega2=np.eye(N)
f_list_kr, v_kr, Q_kr = kr.kernel_regression(data, ret, f_list.copy(), lambda1, lambda2, Omega1, Omega2, 100, N, K)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99


In [11]:
print(metrics.total_R_squared_kr(ret, v_kr, Q_kr))

0.9809999520874729


In [17]:
m_hat = 10
L, B = kr.pivoted_chol(K, m_hat)

1
norm is:  7198.999959104213
2
norm is:  7197.999944784164
3
norm is:  7196.998824179322
4
norm is:  7195.61507336205
5
norm is:  7194.588527986571
6
norm is:  7193.398143087707
7
norm is:  7192.393800172051
8
norm is:  7191.390077186724
9
norm is:  7190.1437715554675
10
norm is:  7188.956930843082
11
norm is:  7187.956060450433
12
norm is:  7186.956060450433
13
norm is:  7185.956060275518
14
norm is:  7184.953919603878
15
norm is:  7183.794209933053
16
norm is:  7182.791290393763
17
norm is:  7181.791253384338
18
norm is:  7180.791218558133
19
norm is:  7179.79121855796
20
norm is:  7178.788475846901
21
norm is:  7177.77408695479
22
norm is:  7176.749253136371
23
norm is:  7175.499858074853
24
norm is:  7174.43878762402
25
norm is:  7173.4387818611685
26
norm is:  7172.438779631779
27
norm is:  7171.191809182346
28
norm is:  7169.980848080299
29
norm is:  7168.975151725551
30
norm is:  7167.965791636323
31
norm is:  7166.952565541898
32
norm is:  7165.947261126903
33
norm is:  7164.8

In [18]:
f_list_kr_LR, v_kr_LR = kr.kernel_regression_LR(data, K, B, ret, f_list.copy(), lambda1, lambda2, Omega2, 100, m_hat, N)

0
1
2
3
4
5
6
7
8
9


KeyboardInterrupt: 

In [ ]:
print(metrics.total_R_squared_kr_LR(ret, B, K, v_kr_LR, f_list_kr_LR))

0.001358626838706134


In [24]:
x_train,y_train,x_test,y_test = split_dataset(data, ret, 0.8)

In [ ]:
def cross_val_Gaussian_LR(y,x, trsh, lambda1_v, lambda2_v, alphas_v, m_hat):

    L, B = kr.pivoted_chol(K, m_hat)

    total_R2_dict = {}
    
    xx_train,yy_train,xx_test,yy_test = split_dataset(x,y, trsh)

    for alpha in alphas_v:

        data2 = xx_train.copy()
        data2 = np.array(np.array(data2).reshape(72*100,94)) #flatten data, build X
        K = kr.K_LR(data2, 1, alpha)

        for lambda1 in lambda1_v:
            for lambda2 in lambda2_v:
            
                
                



                
    return total_R2_dict